In [2]:
import cv2

In [9]:
img=cv2.imread('pic.jpg',1)

# -1 load color pictures any trans
# 0  load grey scale pic
# 1  load picture normal

In [3]:
#resize image
img=cv2.resize(img,(400,400))

In [25]:
#rotating image
img=cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

In [26]:
cv2.imshow('pic',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
#writing to file or output
cv2.imwrite('Image.jpg',img)

True

In [10]:
print(img.shape) #height width and RGB or BGR

(1520, 2048, 3)


# Accessing Camera

In [4]:
import numpy as np
import cv2

In [10]:
cap=cv2.VideoCapture(0)

In [11]:
while True:
    ret, frame= cap.read()   #ret=return
    width=int(cap.get(3))    #3(width) is property of capture object
    height=int(cap.get(4))
    
    image=np.zeros(frame.shape,np.uint8)  #creating blank canvas where i can put image
    smaller_frame=cv2.resize(frame,(0,0),fx=0.5,fy=0.5)
    image[:height//2, :width//2]=smaller_frame #image[:height//2, :width//2]=cv2.rotate(smaller_frame,cv2.ROTATE_180)
    image[height//2:, :width//2]=smaller_frame
    image[:height//2, width//2:]=smaller_frame #image[:height//2, width//2:]=cv2.rotate(smaller_frame,cv2.ROTATE_180)
    image[height//2:, width//2:]=smaller_frame
    
    cv2.imshow('Frame',image)
    if cv2.waitKey(1)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Drawing Lines, Circle, etc

In [50]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    width = int(cap.get(3))
    height = int(cap.get(4))
    
    img=cv2.line(frame, (0,0),(width,height),(255,0,0),10)
    img=cv2.line(img, (0,height),(width,0),(255,0,0),5)
    img=cv2.rectangle(img,(100,100),(200,200),(128,128,128),5)
    img=cv2.circle(img,(300,300),60,(0,0,255),-1)
    font=cv2.FONT_HERSHEY_COMPLEX
    img=cv2.putText(img,'Abhishek',(10,height-10),font,1,(0,0,0),5, cv2.LINE_AA)
    
    cv2.imshow('Frame',frame)
    if cv2.waitKey(1)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Color Detection

In [52]:
import cv2
import numpy
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    width=int(cap.get(3))
    height=int(cap.get(4))
    
    #coverting image from BGR to HSV
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    lower_blue=np.array([90,50,50])
    upper_blue=np.array([130,255,255])
    mask=cv2.inRange(hsv,lower_blue,upper_blue)
    result=cv2.bitwise_and(frame,frame,mask=mask)
    
    cv2.imshow('Frame',result)
    if cv2.waitKey(1)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Corner Detection

In [53]:
import cv2
import numpy
import random

img=cv2.imread('chess.jpg')
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

corners=cv2.goodFeaturesToTrack(gray,100,0.01,10)
corners=np.int0(corners)
for c in corners:
    x,y=c.ravel()
    cv2.circle(img,(x,y),5,(255,0,0),-1)
    
for i in range(len(corners)):
    for j in range(i+1,len(corners)):
        corner1=tuple(corners[i][0])
        corner2=tuple(corners[j][0])
        color=tuple(map(lambda x: int(x),np.random.randint(0,255,size=3)))
        cv2.line(img,corner1,corner2,color,1)
        
        
cv2.imshow("Frame",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Template Matching(Object detection)

In [54]:
import numpy as np
import cv2

In [55]:
img=cv2.resize(cv2.imread('soccer_practice.jpg',0),(0,0),fx=0.8,fy=0.8)
template=cv2.resize(cv2.imread('ball.png',0),(0,0),fx=0.8,fy=0.8)
h,w=template.shape

In [56]:
methods=[cv2.TM_CCOEFF,cv2.TM_CCOEFF_NORMED,cv2.TM_CCORR,cv2.TM_CCORR_NORMED,cv2.TM_SQDIFF,cv2.TM_SQDIFF_NORMED]

In [57]:
for method in methods:
    img2=img.copy()
    
    result=cv2.matchTemplate(img2,template,method)
    min_val,max_val,min_loc,max_loc=cv2.minMaxLoc(result)
    
    #drawing rectangle on the above location
    if method in [cv2.TM_SQDIFF,cv2.TM_SQDIFF_NORMED]:
        location=min_loc
    else:
        location=max_loc
    bottom_right=(location[0]+w,location[1]+h)
    cv2.rectangle(img2,location,bottom_right,255,5)
    cv2.imshow('Match',img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Face and Eye Detection(Using Haarcascade)

In [1]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

face_cascade=cv2.CascadeClassifier('C:\\Users\\user\\anaconda3\\pkgs\\libopencv-4.0.1-hbb9e17c_0\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')
eye_cascade=cv2.CascadeClassifier('C:\\Users\\user\\anaconda3\\pkgs\\libopencv-4.0.1-hbb9e17c_0\\Library\\etc\\haarcascades\\haarcascade_eye.xml')

while True:
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 3)
        roi_gray = gray[y:y+w, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 3)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 3)

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [2]:
print(faces)

[[264 130 151 151]]


In [6]:
import cv2
import numpy as np

In [43]:
cap=cv2.VideoCapture('lol.mp4')
a,b=(cap.read())
#print(a)
#print(b)
#cv2.imshow('Frame',img)
#cv2.waitKey(3000)
#cv2.destroyAllWindows()

# Detect face

In [2]:
import cv2,time
import numpy as np

In [9]:
cap=cv2.VideoCapture(0)
ret,frame=cap.read()
cap.release()